In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import folium 
import random
import numpy as np
from pyomo.environ import *

In [2]:
def load_distance_time_matrix(path, N):
    data= pd.read_csv(path)
    distance = np.zeros((N,N))
    time = np.zeros((N,N))
    
    for i in range(len(data)):
        origen = int(data.iloc[i, 0])
        destino = int(data.iloc[i, 1])
        distance[origen-1, destino-1] = float(data.iloc[i, 2])
        time[origen-1, destino-1] = float(data.iloc[i, 3])
    return distance, time

def load_vehicles(path):
    data= pd.read_csv(path)
    N = len(data)
    vehicles = {}
    for i in range(N):
        id = int(data.iloc[i, 0])
        capacity = int(data.iloc[i, 1])
        ran = float(data.iloc[i, 2])
        vehicles[id] = (capacity, ran)
    
    return vehicles

def load_demand(clientsPath):
    data= pd.read_csv(clientsPath)
    N = len(data)
    demand_dic = {}
    for i in range(N):
        id = int(data.iloc[i, 1])
        demand = float(data.iloc[i, 2])
        demand_dic[id] = demand
    
    return demand_dic


def load_capacity(depositPath):
    data= pd.read_csv(depositPath)
    N = len(data)
    capacity_dic = {}
    for i in range(N):
        id = int(data.iloc[i, 1])
        capacity = float(data.iloc[i, 4])
        capacity_dic[id] = capacity
    
    return capacity_dic

def load_distance_time_dic(path):
    data= pd.read_csv(path)
    distance = {}
    time = {}
    
    for i in range(len(data)):
        origen = int(data.iloc[i, 0])
        destino = int(data.iloc[i, 1])
        distance[origen, destino] = float(data.iloc[i, 2])
        time[origen, destino] = float(data.iloc[i, 3])
    
    return distance, time

def load_coordinates(depotsPath, clientsPath):
    coord = {}
    depot= pd.read_csv(depotsPath)
    client= pd.read_csv(clientsPath)
    
    for i in range(len(depot)):
        id = int(depot.iloc[i, 1])
        lat = float(depot.iloc[i, 3])
        long = float(depot.iloc[i, 2])
        coord[id] = [lat, long]

    for j in range(len(client)):
        id = int(client.iloc[j, 1])
        lat = float(client.iloc[j, 4])
        long = float(client.iloc[j, 3])
        coord[id] = [lat, long]
    
    return coord
        

In [8]:
#Cargar datos
distancia_dic,time_dic = load_distance_time_dic('../Datos/Caso_Base/casoBase.csv')
vehiculos = load_vehicles('../Datos/Caso_Base/vehicles.csv')
demanda = load_demand('..\Datos\Caso_Base\clients.csv')
coord = load_coordinates('..\Datos\Caso_Base\depots.csv', '..\Datos\Caso_Base\clients.csv')

np.random.seed(42)
n = 25  # 24 clientes + 1 depósito

dist_matrix_np, time_matrix = load_distance_time_matrix('..\datos\Caso_Base\casoBase.csv', 25)
# Convertir a diccionario {(i,j): distancia}
distancia = {(i, j): dist_matrix_np[i][j]/1000 for i in range(n) for j in range(n)}
print(f"Distancia: {distancia}")
print(f"Vehiculos: {vehiculos}")
print(f"Demanda: {demanda}")


Distancia: {(0, 0): np.float64(0.0), (0, 1): np.float64(27.144299999999998), (0, 2): np.float64(17.6771), (0, 3): np.float64(13.9796), (0, 4): np.float64(26.6508), (0, 5): np.float64(22.5837), (0, 6): np.float64(18.5625), (0, 7): np.float64(24.765900000000002), (0, 8): np.float64(23.9294), (0, 9): np.float64(27.835900000000002), (0, 10): np.float64(33.6518), (0, 11): np.float64(31.029400000000003), (0, 12): np.float64(16.1247), (0, 13): np.float64(21.6122), (0, 14): np.float64(10.3874), (0, 15): np.float64(27.040200000000002), (0, 16): np.float64(33.9631), (0, 17): np.float64(26.5413), (0, 18): np.float64(20.0532), (0, 19): np.float64(21.278200000000002), (0, 20): np.float64(13.411200000000001), (0, 21): np.float64(27.8695), (0, 22): np.float64(24.2626), (0, 23): np.float64(11.364799999999999), (0, 24): np.float64(30.8098), (1, 0): np.float64(30.809900000000003), (1, 1): np.float64(0.0), (1, 2): np.float64(14.2563), (1, 3): np.float64(19.400299999999998), (1, 4): np.float64(1.1309), (1

In [4]:
from pyomo.environ import *

# Crear el modelo
model = ConcreteModel()

# Datos: nodos, demanda, vehículos, distancias
# Suponiendo que ya tienes definidos:
# - demanda: {cliente_id: valor}
# - vehiculos: {vehiculo_id: (capacidad, rango)}
# - distancia: {(i, j): valor}
# - coord: {nodo: (x, y)} para mostrar rutas al final

nodos = [1] + list(demanda.keys())  # Nodo 1 es el depósito
model.N = Set(initialize=nodos)
model.C = Set(initialize=demanda.keys())
model.V = Set(initialize=vehiculos.keys())
model.D = Set(initialize=[1])  # depósito

# Distancias modificadas sin diagonales
offset = 1  # Si distancia usa índices base 0
dist_modificada = {(i + offset, j + offset): v for (i, j), v in distancia.items() if i != j}

# Conjunto de aristas válidas (sin (i, i))
model.A = Set(dimen=2, initialize=dist_modificada.keys())

# Variables
model.x = Var(model.A, model.V, domain=Binary)  # Ruta tomada por vehículo
model.u = Var(model.N, model.V, domain=NonNegativeReals)  # Carga acumulada

# Parámetros
model.dist = Param(model.A, initialize=dist_modificada, within=PositiveReals)
model.demand = Param(model.C, initialize=demanda)
model.cap = Param(model.V, initialize={v: vehiculos[v][0] for v in vehiculos})
model.range = Param(model.V, initialize={v: vehiculos[v][1] for v in vehiculos})

# Función objetivo
def obj_rule(m):
    return sum(m.dist[i, j] * m.x[i, j, v] for (i, j) in m.A for v in m.V)
model.obj = Objective(rule=obj_rule, sense=minimize)

# Restricciones

# Cada cliente es visitado exactamente una vez
model.visit_once = ConstraintList()
for c in model.C:
    model.visit_once.add(
        sum(model.x[i, c, v] for (i, j) in model.A if j == c for v in model.V) == 1
    )

# Flujo conservado
model.flow_conservation = ConstraintList()
for v in model.V:
    for c in model.C:
        model.flow_conservation.add(
            sum(model.x[i, c, v] for (i, j) in model.A if j == c) ==
            sum(model.x[c, j, v] for (i, j) in model.A if i == c)
        )

# Salida y retorno al depósito por vehículo
model.depot_constraints = ConstraintList()
for v in model.V:
    model.depot_constraints.add(
        sum(model.x[1, j, v] for (i, j) in model.A if i == 1) == 1
    )
    model.depot_constraints.add(
        sum(model.x[i, 1, v] for (i, j) in model.A if j == 1) == 1
    )

# Capacidad
model.capacity_constraint = ConstraintList()
for v in model.V:
    model.capacity_constraint.add(
        sum(model.demand[c] * sum(model.x[i, c, v] for (i, j) in model.A if j == c) for c in model.C) <= model.cap[v]
    )

# Carga inicial en el depósito
model.initial_load = ConstraintList()
for v in model.V:
    model.initial_load.add(model.u[1, v] == 0)

# Carga no excede la capacidad
model.max_capacity = ConstraintList()
for v in model.V:
    for i in model.N:
        model.max_capacity.add(model.u[i, v] <= model.cap[v])

# Subtour elimination (MTZ)
model.subtour_elimination = ConstraintList()
for v in model.V:
    for i in model.C:
        for j in model.C:
            if i != j and (i, j) in model.A:
                model.subtour_elimination.add(
                    model.u[i, v] - model.u[j, v] + model.cap[v] * model.x[i, j, v] <= model.cap[v] - model.demand[j]
                )

# Rango del vehículo
model.range_constraint = ConstraintList()
for v in model.V:
    model.range_constraint.add(
        sum(model.dist[i, j] * model.x[i, j, v] for (i, j) in model.A) <= model.range[v]
    )

# Resolver el modelo
opt = SolverFactory('gurobi')
opt.options['TimeLimit'] = 1200  # Establecer límite de tiempo en segundos
opt.options['MIPGap'] = 0.03  # Establecer tolerancia de optimalidad
opt.options['Threads'] = 4  # Número de hilos a usar
results = opt.solve(model, tee=True)

# Verificar si se encontró una solución factible
if (results.solver.termination_condition == TerminationCondition.maxTimeLimit and
    results.solver.status == SolverStatus.ok):
    print("Tiempo agotado, pero se encontró una solución factible.")

# Acceder a los valores de las variables
for v in model.component_objects(Var, active=True):
    for index in v:
        print(f"{v[index].name} = {value(v[index])}")


# Reconstrucción ordenada de rutas
for v in model.V:
    print(f"\nVehículo {v}:")
    ruta = [1]
    actual = 1
    while True:
        next_nodes = [j for (i, j) in model.A if i == actual and model.x[i, j, v].value > 0.5]
        if not next_nodes:
            break
        siguiente = next_nodes[0]
        ruta.append(siguiente)
        actual = siguiente
        if siguiente == 1:
            break
    print("Ruta:", ruta)

routes = {}

for v in model.V:
    print(f"Vehículo {v}:")
    routes[v] = []

    # Construir lista de arcos activos para el vehículo v
    active_arcs = [(i, j) for i in model.N for j in model.N 
                   if i != j and model.x[i, j, v].value > 0.5]
    
    if not active_arcs:
        continue

    # Iniciar la ruta desde el nodo de depósito (asumimos que es el nodo 1)
    actual = next(i for (i, j) in active_arcs if i in model.D)  # o usar actual = 1 si sabes que siempre empieza ahí
    ruta = [actual]

    while True:
        siguientes = [j for (i, j) in active_arcs if i == actual]
        if not siguientes:
            break
        siguiente = siguientes[0]
        ruta.append(siguiente)
        actual = siguiente
        if siguiente in model.D:
            break

    # Guardar coordenadas ordenadas
    routes[v] = [coord[n] for n in ruta]

    # Mostrar info de carga
    for idx in range(len(ruta) - 1):
        i, j = ruta[idx], ruta[idx+1]
        if i in model.C:
            capacidad_utilizada = model.demand[i] * model.x[i, j, v].value
            print(f"  Ruta {i} -> {j}: Carga transportada = {capacidad_utilizada}")
        else:
            print(f"  Ruta {i} -> {j} (desde depósito)")



Read LP format model from file C:\Users\aleja\AppData\Local\Temp\tmp2afzhgr9.pyomo.lp
Reading time = 0.03 seconds
x1: 4872 rows, 5000 columns, 37072 nonzeros
Set parameter TimeLimit to value 1200
Set parameter MIPGap to value 0.03
Set parameter Threads to value 4
Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (win64 - Windows 11.0 (26100.2))

CPU model: Intel(R) Core(TM) i5-10300H CPU @ 2.50GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 4 threads

Non-default parameters:
TimeLimit  1200
MIPGap  0.03
Threads  4

Optimize a model with 4872 rows, 5000 columns and 37072 nonzeros
Model fingerprint: 0x075d76e1
Variable types: 200 continuous, 4800 integer (4800 binary)
Coefficient statistics:
  Matrix range     [7e-01, 1e+02]
  Objective range  [7e-01, 4e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+02]
Presolve removed 208 rows and 8 columns
Presolve time: 0.05s
Presolved: 4664 rows, 4992 columns, 36712 nonzeros

In [13]:
import re
import pandas as pd

pf = 15000

data = {
    'VehicleId': [],
    'DepotId': [],
    'InitialLoad': [],
    'RouteSequence': [],
    'ClientsServed': [],
    'DemandsSatisfied': [],
    'TotalDistance': [],
    'TotalTime': [],
    'FuelCost': []
}

def isClient(name):
    return isinstance(name, int) and name != 1  # Nodo 1 es el depósito

pf = 15000  # precio por litro de combustible
for v in model.V:
    route_nodes = []
    arcs = [(i, j) for (i, j) in model.A if model.x[i, j, v].value > 0.5]
    
    if not arcs:
        continue
    
    vehicle_id = "VEH" + "{:03d}".format(v)
    data['VehicleId'].append(vehicle_id)

    # Reconstruir la ruta
    actual = 1  # empieza en depósito
    route_nodes = [actual]
    while True:
        next_node = next((j for (i, j) in arcs if i == actual), None)
        if next_node is None or next_node == 1:
            route_nodes.append(1)
            break
        route_nodes.append(next_node)
        actual = next_node

    route_names = ["CD1" if n == 1 else f"C{n}" for n in route_nodes]
    data['DepotId'].append("CD1")
    data['RouteSequence'].append("-".join(route_names))

    # Calcular distancia y tiempo total
    total_dist = sum(model.dist[i, j] for i, j in zip(route_nodes[:-1], route_nodes[1:]))
    total_time = sum(time_dic[(i, j)] for i, j in zip(route_nodes[:-1], route_nodes[1:]))

    data['TotalDistance'].append(round(total_dist / 1000, 1))  # km
    data['TotalTime'].append(round(total_time, 1))             # minutos o lo que corresponda
    data['FuelCost'].append(round((total_dist / 1000) * pf))

    # Clientes servidos y demandas satisfechas
    demands_satisfied = []
    initial_load = 0
    for n in route_nodes:
        if isClient(n):
            d = demanda[n]
            demands_satisfied.append(d)
            initial_load += d

    data['ClientsServed'].append(len(demands_satisfied))
    data['DemandsSatisfied'].append("-".join(str(d) for d in demands_satisfied))
    data['InitialLoad'].append(initial_load)

# Guardar resultados
print(data)
df = pd.DataFrame(data)
df.to_csv("verificacion_caso1.csv", index=False)


{'VehicleId': ['VEH001', 'VEH002', 'VEH003', 'VEH004', 'VEH005', 'VEH006', 'VEH007', 'VEH008'], 'DepotId': ['CD1', 'CD1', 'CD1', 'CD1', 'CD1', 'CD1', 'CD1', 'CD1'], 'InitialLoad': [113.0, 138.0, 15.0, 12.0, 12.0, 15.0, 15.0, 57.0], 'RouteSequence': ['CD1-C7-C19-C5-C2-C16-C23-C14-CD1', 'CD1-C6-C9-C12-C10-C25-C17-C11-C18-CD1', 'CD1-C21-CD1', 'CD1-C13-CD1', 'CD1-C4-CD1', 'CD1-C15-CD1', 'CD1-C24-CD1', 'CD1-C20-C22-C8-C3-CD1'], 'ClientsServed': [7, 8, 1, 1, 1, 1, 1, 4], 'DemandsSatisfied': ['17.0-12.0-15.0-13.0-17.0-18.0-21.0', '20.0-20.0-17.0-20.0-11.0-10.0-15.0-25.0', '15.0', '12.0', '12.0', '15.0', '15.0', '11.0-14.0-17.0-15.0'], 'TotalDistance': [np.float64(0.1), np.float64(0.1), np.float64(0.0), np.float64(0.0), np.float64(0.0), np.float64(0.0), np.float64(0.0), np.float64(0.1)], 'TotalTime': [6337.3, 8886.5, 2956.1, 3351.5, 2953.1, 2542.3, 2733.9, 5573.2], 'FuelCost': [987, 1366, 397, 483, 414, 322, 352, 936]}


In [14]:
m = folium.Map(
    location=[4.743359, -74.153536],
    zoom_start=11,
    tiles='Cartodb Positron' 
)

number_of_colors = 9

colors = ['blue', 'green', 'cyan', 'magenta','olive', 'blue', 'orange', 'purple','red' ]
icons = ['blue', 'green', 'lightblue', 'pink','lightgreen', 'blue', 'orange', 'darkpurple','red' ]

for route in routes.keys():
    if len(routes[route]) != 0 :  
        folium.PolyLine(
            routes[route],
            color=colors[route],
            weight=5,
            opacity=0.7,
            tooltip='Vehículo ' + str(route)
        ).add_to(m)

        folium.Marker(routes[route][0], popup="Inicio", icon=folium.Icon(color='black')).add_to(m)
        folium.Marker(routes[route][-1], popup="Llegada V" + str(route), icon=folium.Icon(color=icons[route])).add_to(m)
m